# 1.Viewing information of Datacube 

In [ ]:
import datacube
dc = datacube.Datacube(app = 'my_app', config = '/home/localuser/.datacube.conf')

list_of_products = dc.list_products()
netCDF_products = list_of_products[list_of_products['format'] == 'NetCDF']
netCDF_products

In [ ]:
#list_of_measurements = dc.list_measurements('ls7_lasrc_taiwan')
list_of_measurements = dc.list_measurements()
list_of_measurements

In [ ]:
import utils.data_access_api as dc_api  
api = dc_api.DataAccessApi(config = '/home/localuser/.datacube.conf')

platform = "LANDSAT_8"
product = "ls8_C1_sr_taiwan"

# Get Coordinates
coordinates = api.get_full_dataset_extent(platform = platform, product = product)
times = coordinates["time"].values 

latitude_extents = (min(coordinates['latitude'].values),max(coordinates['latitude'].values))
print( latitude_extents )
longitude_extents = (min(coordinates['longitude'].values),max(coordinates['longitude'].values))
print( longitude_extents )

# 2.Loading data from Datacube

In [ ]:
import datetime

# define geographic boundaries in (min, max) format
lon = (120.972549, 121.511577)
lat = (24.783987, 25.307047)

In [ ]:
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude = lat, longitude = lon)

In [ ]:
date_range =(datetime.datetime(2015,1,1), datetime.datetime(2015,2,28))
# define product and platform to specify our intent to load Landsat 8 sr products
platform = 'LANDSAT_8'
product = 'ls8_C1_sr_taiwan'  
# define desired bands. 
desired_bands = ['red','green','blue','nir','swir1','swir2','pixel_qa']  

# load area. Should result in approximately 15 acquisitions between 2014 and 2016      
landsat = dc.load(product = product,platform = platform,lat = lat,lon = lon, time = date_range,measurements = desired_bands,group_by = 'solar_day')

In [ ]:
landsat

# 3.Ploting data

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
landsat.green.plot(col='time', col_wrap=5)#.where(landsat.green < 1000).plot(col='time', col_wrap=2)

# 4.Save to tiff

In [ ]:
import time
import rasterio
import os
from dc_utilities import write_geotiff_from_xr

def time_to_string(t):
    return time.strftime("%Y_%m_%d_%H_%M_%S", time.gmtime(t.astype(int)/1000000000))

def export_slice_to_geotiff(ds, path):
    write_geotiff_from_xr(path,ds.astype(np.float32),list(ds.data_vars.keys()),crs="EPSG:4326")

#For each time slice in a dataset we call export_slice_to_geotif
def export_xarray_to_geotiff(ds, path):
    for t in ds.time:
        time_slice_xarray = ds.sel(time = t)
        export_slice_to_geotiff(time_slice_xarray, path + "_" + time_to_string(t) + ".tif")
#Start Export
output_dir = "~/data"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

export_xarray_to_geotiff(combined_dataset, "{}/{}".format(output_dir,product))

In [ ]:
from utils.dc_utilities import write_geotiff_from_xr
import numpy as np
write_geotiff_from_xr("../test.tiff",landsat.sel(time = landsat.time[1]).astype(np.float32),list(landsat.sel(time = landsat.time[1]).data_vars.keys()),crs="EPSG:4326")

# 5.Save to NetCDF

In [ ]:
import xarray as xr
#xr.Dataset(landsat.values(), coords = landsat.coords, dims = landsat.dims)
temp = list(landsat.data_vars.keys())
for i in temp:
    landsat[i].attrs = []
landsat.to_netCDF('test.nc')